In [1]:
import collections
import pandas as pd
import pyreadr
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from venn import venn
import numpy as np
import math
import scipy

/var/folders/y8/66w8_8kj0dx2znq80cdbl_rw0000gn/T/ipykernel_45674/2185171821.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
clinical_df = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/figurecode/data/20221109_TRACERx421_all_patient_df.rds')
clinical_df = pd.DataFrame(clinical_df[None])
clinical_df.head()

,cruk_id,tumour_id_muttable_cruk,tumour_id_per_patient,age,sex,ethnicity,cigs_perday,years_smoking,packyears,smoking_status_merged,...,os_time,cens_dfs,dfs_time,cens_dfs_any_event,dfs_time_any_event,cens_lung_event,lung_event_time,Relapse_cat,Relapse_cat_new,tx100
0,CRUK0034,CRUK0034,CRUK0034,68,Female,White- Irish,20.0,35,35.000,Ex-Smoker,...,1849,0,1849,0,1849,0,1849,NaN,No rec,True
1,CRUK0150,CRUK0150,CRUK0150,81,Male,White- British,44.5,49,109.025,Ex-Smoker,...,1362,1,1362,1,1362,0,1362,NaN,No rec,False
2,CRUK0159,CRUK0159,CRUK0159,60,Male,White- British,20.0,38,38.000,Smoker,...,2224,1,2224,1,1935,0,2224,Second primary lung,No rec,False
3,CRUK0090,CRUK0090,CRUK0090,65,Male,White- British,10.0,35,17.500,Ex-Smoker,...,2365,1,1143,1,1143,1,1143,Intrathoracic,Intrathoracic,True
4,CRUK0045,CRUK0045,CRUK0045,85,Male,White- British,10.0,25,12.500,Ex-Smoker,...,986,1,917,1,917,1,917,Intrathoracic,Intrathoracic,True


In [3]:
mut_table = pyreadr.read_r('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/LCCE_converted_data/20221109_TRACERx421_mutation_table.rds')
mut_table = pd.DataFrame(mut_table[None])
mut_table.head()

,patient_id,tumour_id,mutation_id,chr,start,stop,ref,var,Hugo_Symbol,func,...,AAChange,GL_VAF,GL_nAlt,GL_depth,ITHState,PyCloneClonal_SC,PyCloneCluster_SC,cleanCluster_SC,combTiming_SC,DriverMut
0,CRUK0005,CRUK0005,CRUK0005:19:34291428:G,chr19,34291428,34291428,G,T,KCTD15,intronic,...,NaN,0,0,187,1,S,3.0,True,subclonal,False
1,CRUK0005,CRUK0005,CRUK0005:19:9084770:C,chr19,9084770,9084770,C,A,MUC16,exonic,...,p.V2349L,0.39,2,510,1,C,2.0,True,early,False
2,CRUK0005,CRUK0005,CRUK0005:1:2160418:G,chr1,2160418,2160418,G,T,SKI,exonic,...,p.P71P,0,0,114,1,C,2.0,True,early,False
3,CRUK0005,CRUK0005,CRUK0005:19:40095981:G,chr19,40095981,40095981,G,T,LGALS13,exonic,...,p.G86C,0,0,659,1,C,2.0,True,early,False
4,CRUK0005,CRUK0005,CRUK0005:19:41313703:C,chr19,41313703,41313703,C,G,RAB4B-EGLN2,ncRNA_intronic,...,NaN,0,0,894,3,S,5.0,True,subclonal,False


In [4]:
clinical_df = clinical_df.sort_values(by=['cruk_id'])
clinical_df

,cruk_id,tumour_id_muttable_cruk,tumour_id_per_patient,age,sex,ethnicity,cigs_perday,years_smoking,packyears,smoking_status_merged,...,os_time,cens_dfs,dfs_time,cens_dfs_any_event,dfs_time_any_event,cens_lung_event,lung_event_time,Relapse_cat,Relapse_cat_new,tx100
88,CRUK0001,CRUK0001,CRUK0001,84,Male,White- British,20.0,30,30.00,Ex-Smoker,...,1120,1,1120,1,1120,0,1120,NaN,No rec,True
79,CRUK0002,CRUK0002,CRUK0002,81,Male,White- British,5.0,35,8.75,Ex-Smoker,...,1901,0,1901,0,1901,0,1901,NaN,No rec,True
26,CRUK0003,CRUK0003,CRUK0003,78,Female,White- British,0.0,0,0.00,Never Smoked,...,2304,1,217,1,217,1,217,Intra & Extra,Intra & Extra,True
23,CRUK0004,CRUK0004,CRUK0004,69,Female,White- British,10.0,27,13.50,Ex-Smoker,...,1323,1,325,1,325,1,325,Intra & Extra,Intra & Extra,True
7,CRUK0005,CRUK0005,CRUK0005,74,Female,White- British,20.0,56,56.00,Ex-Smoker,...,649,1,649,1,649,0,649,NaN,No rec,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414,CRUK0861,CRUK0861,CRUK0861,65,Female,Middle eastern,10.0,45,22.50,Smoker,...,1084,0,1084,0,1084,0,1084,NaN,No rec,False
418,CRUK0864,CRUK0864,CRUK0864,78,Female,White- British,10.0,30,15.00,Ex-Smoker,...,1434,0,1434,0,1434,0,1434,NaN,No rec,False
416,CRUK0872,CRUK0872,CRUK0872,66,Male,White- British,20.0,54,54.00,Ex-Smoker,...,893,1,883,1,883,1,883,Intra & Extra,Intra & Extra,False
420,CRUK0881,CRUK0881_Tumour1,CRUK0881_Tumour1;CRUK0881_Tumour2,84,Male,White- British,20.0,70,70.00,Smoker,...,758,1,748,1,748,1,748,Intrathoracic,Intrathoracic,False


In [5]:
# Making the training data

y_train = []
for index, row in clinical_df.iterrows():
    if row['first_dfs_any_event_rec.or.new.primary'] == 'recurrence':
        y_train.append(1)
    else:
        y_train.append(0)

y = np.array(y_train)
print(y)

[0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1 1 1 1
 1 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 1 0
 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 1
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 0 0 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 0 1
 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 1 0
 0 0 0 1 1 0 1 0 1 0 1 0 1 1 0 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
 1 1 0 0 1 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 1
 0 1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 1 0 0 1 1 1 0 1
 0 0 0 1 0 0 0 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 0 1 0
 1 0 0 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1
 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 1 0 0 0 1
 0 1 0 0 0 0 0 1 1 0 0 1 1 0]


In [6]:
gene_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/100_gene_matrix.csv')
driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/100_driver_matrix.csv')
fifty_gene_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/50_gene_matrix.csv')
fifty_driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/50_driver_matrix.csv')
twenty_gene_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/20_gene_matrix.csv')
twenty_driver_matrix = pd.read_csv('/Users/jasminechau/PycharmProjects/pythonProject1/INIM0038/Matrices/20_driver_matrix.csv')

In [2]:
# P vs M
PvM_lung = ['TP53', 'CDKN2A', 'NKX2-1', 'EGFR', 'MYC', 'FOXA1', 'KEAP1', 'TERT', 'SMARCA4', 'ALK']
PvM_pancancer = ['TP53', 'CDKN2A', 'NKX2-1', 'EGFR', 'MYC', 'FOXA1', 'KEAP1', 'TERT', 'SMARCA4', 'ALK', 'PTEN', 'APC', 'CTNNB1', 'CDK12', 'AR', 'RB1', 'NKX3-1', 'CCND1', 'FGFR1', 'ARID1A', 'AKT1', 'KIT', 'BCL2L1', 'DNMT3B', 'CDKN1B', 'PAK1', 'ESR1', 'SRC', 'SMAD4', 'ERBB2', 'MDM4', 'KRAS', 'SETD2']

# P-noM vs P-M
PnoMvPM_lung = ['TP53', 'CDKN2A', 'NKX2-1', 'EGFR', 'FOXA1', 'KEAP1', 'SMARCA4']
PnoMvPM_pancancer = ['TP53', 'CDKN2A', 'NKX2-1', 'EGFR', 'FOXA1', 'KEAP1', 'SMARCA4', 'CCND1', 'FGFR1', 'ARID1A', 'MDM2', 'PTEN', 'CDK12', 'MYC', 'TERT']

# Mutations correlated to metastatic burden
mut_Met_lung = ['TP53', 'CDKN2A', 'EGFR', 'MYC']
mut_Met_pancancer = ['TP53', 'CDKN2A', 'EGFR', 'MYC', 'AR', 'PTEN', 'ESR1', 'KRAS', 'SMAD4', 'TERT', 'RMB10', 'CCND1', 'FGF19']

In [3]:
len(PvM_lung), len(PvM_pancancer), len(PnoMvPM_lung), len(PnoMvPM_pancancer), len(mut_Met_lung), len(mut_Met_pancancer)

(10, 33, 7, 15, 4, 13)

In [8]:
patient_id = [id for id in clinical_df['cruk_id']]
patient_id

['CRUK0001',
 'CRUK0002',
 'CRUK0003',
 'CRUK0004',
 'CRUK0005',
 'CRUK0006',
 'CRUK0007',
 'CRUK0008',
 'CRUK0009',
 'CRUK0010',
 'CRUK0011',
 'CRUK0012',
 'CRUK0013',
 'CRUK0014',
 'CRUK0015',
 'CRUK0016',
 'CRUK0017',
 'CRUK0018',
 'CRUK0019',
 'CRUK0020',
 'CRUK0021',
 'CRUK0022',
 'CRUK0023',
 'CRUK0024',
 'CRUK0025',
 'CRUK0026',
 'CRUK0027',
 'CRUK0028',
 'CRUK0029',
 'CRUK0030',
 'CRUK0032',
 'CRUK0033',
 'CRUK0034',
 'CRUK0035',
 'CRUK0036',
 'CRUK0037',
 'CRUK0038',
 'CRUK0039',
 'CRUK0040',
 'CRUK0041',
 'CRUK0042',
 'CRUK0043',
 'CRUK0044',
 'CRUK0045',
 'CRUK0046',
 'CRUK0047',
 'CRUK0048',
 'CRUK0049',
 'CRUK0050',
 'CRUK0051',
 'CRUK0052',
 'CRUK0053',
 'CRUK0054',
 'CRUK0055',
 'CRUK0056',
 'CRUK0057',
 'CRUK0058',
 'CRUK0059',
 'CRUK0060',
 'CRUK0061',
 'CRUK0062',
 'CRUK0063',
 'CRUK0064',
 'CRUK0065',
 'CRUK0066',
 'CRUK0067',
 'CRUK0068',
 'CRUK0069',
 'CRUK0070',
 'CRUK0071',
 'CRUK0072',
 'CRUK0073',
 'CRUK0074',
 'CRUK0075',
 'CRUK0076',
 'CRUK0077',
 'CRUK0078',

In [9]:
# Filter the original DataFrame based on the selected genes
filtered_mut_table = mut_table[mut_table['Hugo_Symbol'].isin(PvM_lung)]

# Create a binary matrix (presence or absence of genes for each patient)
PvM_lung_matrix = pd.crosstab(filtered_mut_table['patient_id'], filtered_mut_table['Hugo_Symbol'], values=filtered_mut_table['DriverMut'].astype(int), aggfunc='max').clip(upper=1).fillna(0)

# Add back in rows where all genes are absent
for id in patient_id:
    if id not in PvM_lung_matrix.index:
        PvM_lung_matrix.loc[id] = 0

PvM_lung_matrix = PvM_lung_matrix.sort_index()
PvM_lung_matrix = PvM_lung_matrix.astype(int)

# Display the resulting gene matrix
print(PvM_lung_matrix.shape)
print(PvM_lung_matrix)

(421, 10)
Hugo_Symbol  ALK  CDKN2A  EGFR  FOXA1  KEAP1  MYC  NKX2-1  SMARCA4  TERT  TP53
patient_id                                                                    
CRUK0001       0       0     0      0      0    0       0        0     0     1
CRUK0002       0       0     0      0      0    0       0        0     0     0
CRUK0003       0       0     1      0      0    0       0        0     0     0
CRUK0004       0       0     1      0      0    0       0        0     0     1
CRUK0005       0       0     0      0      0    0       0        0     0     1
...          ...     ...   ...    ...    ...  ...     ...      ...   ...   ...
CRUK0861       0       0     0      0      0    0       0        0     0     1
CRUK0864       0       0     0      0      0    0       0        0     0     0
CRUK0872       0       0     0      0      0    0       0        0     0     1
CRUK0881       0       0     0      0      0    0       0        0     0     1
CRUK0893       0       0     0      0     

In [10]:
# Filter the original DataFrame based on the selected genes
filtered_mut_table = mut_table[mut_table['Hugo_Symbol'].isin(PvM_pancancer)]

# Create a binary matrix (presence or absence of genes for each patient)
PvM_pancancer_matrix = pd.crosstab(filtered_mut_table['patient_id'], filtered_mut_table['Hugo_Symbol'], values=filtered_mut_table['DriverMut'].astype(int), aggfunc='max').clip(upper=1).fillna(0)

# Add back in rows where all genes are absent
for id in patient_id:
    if id not in PvM_pancancer_matrix.index:
        PvM_pancancer_matrix.loc[id] = 0

PvM_pancancer_matrix = PvM_pancancer_matrix.sort_index()
PvM_pancancer_matrix = PvM_pancancer_matrix.astype(int)

# Display the resulting gene matrix
print(PvM_pancancer_matrix.shape)
print(PvM_pancancer_matrix)

(421, 33)
Hugo_Symbol  AKT1  ALK  APC  AR  ARID1A  BCL2L1  CCND1  CDK12  CDKN1B  CDKN2A  \
patient_id                                                                      
CRUK0001        0    0    0   0       0       0      0      0       0       0   
CRUK0002        0    0    0   0       0       0      0      0       0       0   
CRUK0003        0    0    0   0       0       0      0      0       0       0   
CRUK0004        0    0    0   0       0       0      0      0       0       0   
CRUK0005        0    0    0   0       0       0      0      0       0       0   
...           ...  ...  ...  ..     ...     ...    ...    ...     ...     ...   
CRUK0861        0    0    0   0       0       0      0      0       0       0   
CRUK0864        0    0    0   0       0       0      0      0       0       0   
CRUK0872        0    0    0   0       0       0      0      0       0       0   
CRUK0881        0    0    0   0       0       0      0      0       0       0   
CRUK0893        0 

In [11]:
# Filter the original DataFrame based on the selected genes
filtered_mut_table = mut_table[mut_table['Hugo_Symbol'].isin(PnoMvPM_lung)]

# Create a binary matrix (presence or absence of genes for each patient)
PnoMvPM_lung_matrix = pd.crosstab(filtered_mut_table['patient_id'], filtered_mut_table['Hugo_Symbol'], values=filtered_mut_table['DriverMut'].astype(int), aggfunc='max').clip(upper=1).fillna(0)

# Add back in rows where all genes are absent
for id in patient_id:
    if id not in PnoMvPM_lung_matrix.index:
        PnoMvPM_lung_matrix.loc[id] = 0

PnoMvPM_lung_matrix = PnoMvPM_lung_matrix.sort_index()
PnoMvPM_lung_matrix = PnoMvPM_lung_matrix.astype(int)

# Display the resulting gene matrix
print(PnoMvPM_lung_matrix.shape)
print(PnoMvPM_lung_matrix)

(421, 7)
Hugo_Symbol  CDKN2A  EGFR  FOXA1  KEAP1  NKX2-1  SMARCA4  TP53
patient_id                                                    
CRUK0001          0     0      0      0       0        0     1
CRUK0002          0     0      0      0       0        0     0
CRUK0003          0     1      0      0       0        0     0
CRUK0004          0     1      0      0       0        0     1
CRUK0005          0     0      0      0       0        0     1
...             ...   ...    ...    ...     ...      ...   ...
CRUK0861          0     0      0      0       0        0     1
CRUK0864          0     0      0      0       0        0     0
CRUK0872          0     0      0      0       0        0     1
CRUK0881          0     0      0      0       0        0     1
CRUK0893          0     0      0      0       0        0     0

[421 rows x 7 columns]


In [12]:
# Filter the original DataFrame based on the selected genes
filtered_mut_table = mut_table[mut_table['Hugo_Symbol'].isin(PnoMvPM_pancancer)]

# Create a binary matrix (presence or absence of genes for each patient)
PnoMvPM_pancancer_matrix = pd.crosstab(filtered_mut_table['patient_id'], filtered_mut_table['Hugo_Symbol'], values=filtered_mut_table['DriverMut'].astype(int), aggfunc='max').clip(upper=1).fillna(0)

# Add back in rows where all genes are absent
for id in patient_id:
    if id not in PnoMvPM_pancancer_matrix.index:
        PnoMvPM_pancancer_matrix.loc[id] = 0

PnoMvPM_pancancer_matrix = PnoMvPM_pancancer_matrix.sort_index()
PnoMvPM_pancancer_matrix = PnoMvPM_pancancer_matrix.astype(int)

# Display the resulting gene matrix
print(PnoMvPM_pancancer_matrix.shape)
print(PnoMvPM_pancancer_matrix)

(421, 15)
Hugo_Symbol  ARID1A  CCND1  CDK12  CDKN2A  EGFR  FGFR1  FOXA1  KEAP1  MDM2  \
patient_id                                                                   
CRUK0001          0      0      0       0     0      0      0      0     0   
CRUK0002          0      0      0       0     0      0      0      0     0   
CRUK0003          0      0      0       0     1      0      0      0     0   
CRUK0004          0      0      0       0     1      0      0      0     0   
CRUK0005          0      0      0       0     0      0      0      0     0   
...             ...    ...    ...     ...   ...    ...    ...    ...   ...   
CRUK0861          0      0      0       0     0      0      0      0     0   
CRUK0864          0      0      0       0     0      0      0      0     0   
CRUK0872          0      0      0       0     0      0      0      0     0   
CRUK0881          0      0      0       0     0      0      0      0     0   
CRUK0893          0      0      0       0     0      0

In [13]:
# Filter the original DataFrame based on the selected genes
filtered_mut_table = mut_table[mut_table['Hugo_Symbol'].isin(mut_Met_lung)]

# Create a binary matrix (presence or absence of genes for each patient)
mut_Met_lung_matrix = pd.crosstab(filtered_mut_table['patient_id'], filtered_mut_table['Hugo_Symbol'], values=filtered_mut_table['DriverMut'].astype(int), aggfunc='max').clip(upper=1).fillna(0)

# Add back in rows where all genes are absent
for id in patient_id:
    if id not in mut_Met_lung_matrix.index:
        mut_Met_lung_matrix.loc[id] = 0

mut_Met_lung_matrix = mut_Met_lung_matrix.sort_index()
mut_Met_lung_matrix = mut_Met_lung_matrix.astype(int)

# Display the resulting gene matrix
print(mut_Met_lung_matrix.shape)
print(mut_Met_lung_matrix)

(421, 4)
Hugo_Symbol  CDKN2A  EGFR  MYC  TP53
patient_id                          
CRUK0001          0     0    0     1
CRUK0002          0     0    0     0
CRUK0003          0     1    0     0
CRUK0004          0     1    0     1
CRUK0005          0     0    0     1
...             ...   ...  ...   ...
CRUK0861          0     0    0     1
CRUK0864          0     0    0     0
CRUK0872          0     0    0     1
CRUK0881          0     0    0     1
CRUK0893          0     0    0     0

[421 rows x 4 columns]


In [14]:
# Filter the original DataFrame based on the selected genes
filtered_mut_table = mut_table[mut_table['Hugo_Symbol'].isin(mut_Met_pancancer)]

# Create a binary matrix (presence or absence of genes for each patient)
mut_Met_pancancer_matrix = pd.crosstab(filtered_mut_table['patient_id'], filtered_mut_table['Hugo_Symbol'], values=filtered_mut_table['DriverMut'].astype(int), aggfunc='max').clip(upper=1).fillna(0)

# Add back in rows where all genes are absent
for id in patient_id:
    if id not in mut_Met_pancancer_matrix.index:
        mut_Met_pancancer_matrix.loc[id] = 0

mut_Met_pancancer_matrix = mut_Met_pancancer_matrix.sort_index()
mut_Met_pancancer_matrix = mut_Met_pancancer_matrix.astype(int)

# Display the resulting gene matrix
print(mut_Met_pancancer_matrix.shape)
print(mut_Met_pancancer_matrix)

(421, 12)
Hugo_Symbol  AR  CCND1  CDKN2A  EGFR  ESR1  FGF19  KRAS  MYC  PTEN  SMAD4  \
patient_id                                                                  
CRUK0001      0      0       0     0     0      0     0    0     0      0   
CRUK0002      0      0       0     0     0      0     0    0     0      0   
CRUK0003      0      0       0     1     0      0     0    0     0      0   
CRUK0004      0      0       0     1     0      0     0    0     0      1   
CRUK0005      0      0       0     0     0      0     0    0     0      0   
...          ..    ...     ...   ...   ...    ...   ...  ...   ...    ...   
CRUK0861      0      0       0     0     0      0     0    0     0      0   
CRUK0864      0      0       0     0     0      0     1    0     0      0   
CRUK0872      0      0       0     0     0      0     0    0     0      0   
CRUK0881      0      0       0     0     0      0     1    0     0      0   
CRUK0893      0      0       0     0     0      0     1    0     0

In [15]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(PvM_lung_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 10) (169, 10) (252,) (169,)
Best parameters:  {'alpha': 0.1, 'hidden_layer_sizes': (15,)}
Best score:  0.5320855614973261
Predictions:  [0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5260025062656641
ROC AUC:  0.5105733082706767
               precision    recall  f1-score   support

No Recurrence       0.68      0.96   

In [16]:
X_train, X_test, y_train, y_test = train_test_split(PvM_pancancer_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 33) (169, 33) (252,) (169,)
Best parameters:  {'alpha': 0.01, 'hidden_layer_sizes': (5,)}
Best score:  0.5488413547237077
Predictions:  [0 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 1 0 1 0 1 0 0 1 0 1 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5292136591478697
ROC AUC:  0.5205983709273183
               precision    recall  f1-score   support

No Recurrence       0.68      0.83   

In [17]:
X_train, X_test, y_train, y_test = train_test_split(PnoMvPM_lung_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 7) (169, 7) (252,) (169,)
Best parameters:  {'alpha': 0.01, 'hidden_layer_sizes': (5,)}
Best score:  0.5321746880570409
Predictions:  [0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5260025062656641
ROC AUC:  0.49741541353383456
               precision    recall  f1-score   support

No Recurrence       0.68      0.96    

In [18]:
X_train, X_test, y_train, y_test = train_test_split(PnoMvPM_pancancer_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 15) (169, 15) (252,) (169,)
Best parameters:  {'alpha': 0.001, 'hidden_layer_sizes': (15,)}
Best score:  0.5255793226381462
Predictions:  [0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5169172932330828
ROC AUC:  0.45895989974937346
               precision    recall  f1-score   support

No Recurrence       0.67      0.93

In [19]:
X_train, X_test, y_train, y_test = train_test_split(mut_Met_lung_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

(252, 4) (169, 4) (252,) (169,)
Best parameters:  {'alpha': 1e-05, 'hidden_layer_sizes': (10,)}
Best score:  0.5117647058823529
Predictions:  [0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5086152882205514
ROC AUC:  0.46812343358395986
               precision    recall  f1-score   support

No Recurrence       0.67      0.98  

In [20]:
X_train, X_test, y_train, y_test = train_test_split(mut_Met_pancancer_matrix, y, test_size = 0.4, random_state = 0, stratify = y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)


param_grid = {
    'hidden_layer_sizes': [(1,), (2,), (5,), (10,), (15,)],
    'alpha': [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 10, 100]
}

clf = MLPClassifier(solver='lbfgs', random_state=1, max_iter=2000)

grid_search = GridSearchCV(clf, param_grid, scoring='balanced_accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

predictions = grid_search.predict(X_test)

target_names = ['No Recurrence', 'Recurrence']

y_score = grid_search.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_score)

print("Predictions: ",predictions)
print("Actual: ",y_test)
print("Balanced Accuracy: ",balanced_accuracy_score(y_test, predictions))
print("ROC AUC: ",roc_auc)
print(classification_report(y_test, predictions, target_names=target_names))

# fpr, tpr, thresholds = roc_curve(y_test, y_score, pos_label=1)

# plt.figure()
# plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.0])
# plt.xlabel('False Positive Rate or (1 - Specifity)')
# plt.ylabel('True Positive Rate or (Sensitivity)')
# plt.title('Receiver Operating Characteristic')
# plt.legend(loc="lower right")
# plt.show()

(252, 12) (169, 12) (252,) (169,)


Best parameters:  {'alpha': 1, 'hidden_layer_sizes': (15,)}
Best score:  0.5145276292335115
Predictions:  [0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual:  [0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 0
 0 0 0 0 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1
 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 1 0 1 0 1 0 0 1 0 1 0 0 0 0]
Balanced Accuracy:  0.5043076441102757
ROC AUC:  0.4928728070175439
               precision    recall  f1-score   support

No Recurrence       0.66      0.99      0.80       112
   Recurrence     